**Boom. Let's break the augmentation myth open.**  
Not just what augmentations *are*, but which ones actually help — and how much.

---

# 🧪 `08_lab_data_augmentation_comparison.ipynb`  
### 📁 `02_computer_vision`  
> Try common image augmentations (flip, rotate, crop, cutout, mixup)  
> Compare their **impact on accuracy**, overfitting, and training curves.

---

## 💻 Setup Targets

| Spec                    | Design    |
|-------------------------|-----------|
| Platform                | ✅ Colab / CPU / T4 GPU  
| Dataset                 | ✅ CIFAR-10 (small & fast)  
| Epochs                  | 🔁 5–10 max (quick comparisons)  
| Model                   | ✅ Tiny CNN / ResNet18  
| Augmentations           | 🔄 TorchVision `transforms`  
| Visualization           | ✅ Matplotlib  

---

## 🔧 Section 1: Imports & Setup

```python
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
```

---

## 🗂️ Section 2: Define Augmentation Variants

```python
basic = transforms.Compose([
    transforms.ToTensor()
])

flip = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

crop = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])

cutout = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomErasing(p=1.0, scale=(0.1, 0.2))
])

all_combo = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5)
])
```

---

## 📦 Section 3: Load CIFAR with Different Transforms

```python
def get_loader(transform):
    dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=transform)
    return torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
```

---

## 🧠 Section 4: Define a Simple CNN

```python
import torch.nn as nn
import torch.nn.functional as F

class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)
```

---

## 🏃‍♂️ Section 5: Train Function

```python
def train_model(trainloader):
    model = TinyCNN()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    losses = []
    for epoch in range(5):
        total_loss = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        losses.append(total_loss)
    return losses
```

---

## 📈 Section 6: Compare Augmentations

```python
transform_sets = {
    "basic": basic,
    "flip": flip,
    "crop": crop,
    "cutout": cutout,
    "combo": all_combo
}

loss_results = {}

for name, tf in transform_sets.items():
    print(f"Training with: {name}")
    loader = get_loader(tf)
    loss_results[name] = train_model(loader)
```

---

## 📊 Section 7: Plot Loss Curves

```python
for name, losses in loss_results.items():
    plt.plot(losses, label=name)
plt.title("Training Loss by Augmentation Strategy")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()
```

---

## ✅ Wrap-Up Takeaways

| Augmentation | Why It Helps |
|--------------|--------------|
| Flip         | Avoid overfitting directionally-bias patterns  
| Crop         | Makes model robust to spatial shifts  
| Cutout       | Simulates occlusion, encourages abstraction  
| Combo        | Most generalization power in one pass  

---

## 🧠 What You Learned

- **Not all augmentations are equal** — some help more than others  
- **Combo augmentation** can significantly improve generalization  
- You now know how to **test, track, and prove** what works

---

## ✅ Colab-Ready Checklist

| ✅ Feature                 | Status |
|----------------------------|--------|
| Low VRAM                  | ✅      |
| Quick training            | ✅      |
| Visualization of results  | ✅      |
| Scalable to any transform | ✅      |

---

Ready to jump into the next lab?  
`09_lab_finetune_resnet_on_custom_data.ipynb` is next — we’ll fine-tune a real ResNet on your own dataset or a subset of CIFAR/flowers. Let’s go?